In [1]:
import requests
import xml.etree.ElementTree as ET
import random
import pandas as pd

In [2]:
# Минимум и максимум длины каждой случайной последовательности.
sequence_min_length = 300
sequence_max_length = 1000
# Отсюда будем брать последовательности.
genome_url_format = "https://genome.ucsc.edu/cgi-bin/das/hg38/dna?segment=chr{chromosome}:{begin},{end}"

In [3]:
# Длины хромосом.
chromosome_lengths = {"1": 248956422,
                      "2": 242193529,
                      "3": 198295559,
                      "4": 190214555,
                      "5": 181538259,
                      "6": 170805979,
                      "7": 159345973,
                      "8": 145138636,
                      "9": 138394717,
                      "10": 133797422,
                      "11": 135086622,
                      "12": 133275309,
                      "13": 114364328,
                      "14": 107043718,
                      "15": 101991189,
                      "16": 90338345,
                      "17": 83257441,
                      "18": 80373285,
                      "19": 58617616,
                      "20": 64444167,
                      "21": 46709983,
                      "22": 50818468,
                      "X": 156040895,
                      "Y": 57227415}

In [4]:
# Получает случайный регион в геноме.
def get_random_genome_region(sequence_length):
    chromosome = random.choice(list(chromosome_lengths.keys()))
    begin = random.randint(1, chromosome_lengths[chromosome] - sequence_length)
    end = begin + sequence_length - 1

    return chromosome, begin, end

In [5]:
# Получает последовательность из генома.
def get_genome_sequence(chromosome: str, begin: int, end: int) -> str:    
    url = genome_url_format.format(chromosome=chromosome, begin=begin, end=end)
    r = requests.get(url)
    if r.status_code != 200:
        assert False, "Failed to open genome url"

    xml_root = ET.fromstring(r.text)
    sequence = xml_root[0][0]
    
    return sequence.text

In [7]:
sequence_header_format = ">{chromosome}|{begin}|{end} "

In [36]:
def write_fasta_file(output_file):
    with open(output_file, "w") as f:
        for i in range(100):
            print(f"Getting sequence {i + 1}...")
            
            while True:
                chromosome, begin, end = get_random_genome_region(random.randint(sequence_min_length, sequence_max_length))
                print(chromosome, begin, end)
                sequence = get_genome_sequence(chromosome, begin, end).upper()

                # Проверяем, чтобы было только A/T/G/C.
                restart = False
                for ch in sequence:
                    if ch not in "ATGC\n":
                        restart = True
                        break
                if restart:
                    continue                

                break
            
            f.writelines([sequence_header_format.format(chromosome=chromosome, begin=begin, end=end),
                          sequence])

In [37]:
write_fasta_file("./files/human-genome-blast-output.fasta") 

Getting sequence 1...
7 129522162 129522921
Getting sequence 2...
6 84090404 84091171
Getting sequence 3...
22 38742410 38743304
Getting sequence 4...
12 95707023 95707883
Getting sequence 5...
Y 45388289 45388970
3 154064423 154065166
Getting sequence 6...
6 97921464 97922227
Getting sequence 7...
Y 6070107 6070634
Getting sequence 8...
18 9760783 9761243
Getting sequence 9...
5 96949574 96950343
Getting sequence 10...
17 61350803 61351250
Getting sequence 11...
19 15112428 15113174
Getting sequence 12...
20 10297752 10298580
Getting sequence 13...
8 77329437 77330432
Getting sequence 14...
20 14478947 14479637
Getting sequence 15...
9 68284317 68284783
Getting sequence 16...
13 104940897 104941713
Getting sequence 17...
5 44474604 44475016
Getting sequence 18...
5 31034824 31035154
Getting sequence 19...
10 57928597 57929572
Getting sequence 20...
10 36637082 36637759
Getting sequence 21...
7 138763170 138763977
Getting sequence 22...
1 229456710 229457061
Getting sequence 23...
7 10

In [6]:

def avg_identity(input_file):
    df = pd.read_csv(input_file, header=None)
    return df[2].mean()

In [8]:
print(avg_identity('./files/blast-result.csv'))

84.26870454210898


### Код для задание 2: Фрагментация ДНК

In [6]:
chromosome, start, end = get_random_genome_region(100)
seq = get_genome_sequence(chromosome, start, end).replace("\n", "")
print(chromosome, start, end)
print(seq)

4 166802003 166802102
gttcaattgtgagatatgttttcgacacagagccaagtcagaaagtttttcccaaaaaattaaaaaatatatccatcactgtcttgaggtagacaataca


In [7]:
length = len(seq.replace("\n", ""))
with open("./files/fragment-dna.fasta", "w") as f:    
    for i in range(0, 100):
        f.writelines([f">chr={chromosome}|start={start}|end={end - i}|length={length - i} \n", 
                      seq[:length - i] + "\n"])